# Cleaned & Optimized Notebook

**File:** `damage_road_model_cleaned.ipynb`

**Notes:**
- This cleaned version removes duplicate/debug cells and provides a slightly deeper, well-commented CNN.
- I assumed image size **224×224** (common default). If your original notebook used a different size, change `IMG_SIZE` in the 'Constants' cell.

---

## Step 1 — Mount Google Drive & Extract dataset

If you're running locally, adapt paths accordingly.

In [3]:
from pathlib import Path
import zipfile

# 🔹 UPDATE this path to where your ZIP file is stored locally
ZIP_PATH = Path(r"C:\Users\Hardik Agarwal\Downloads\archive (9).zip")  # 👈 yaha apne zip ka actual path daalna

# 🔹 Folder where dataset will be extracted
EXTRACT_DIR = Path("dataset_extracted")

# 🔹 Extraction logic
if ZIP_PATH.exists() and not EXTRACT_DIR.exists():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(EXTRACT_DIR)
    print('✅ Extracted to:', EXTRACT_DIR.resolve())
else:
    print('⚠️ Either ZIP_PATH does not exist or EXTRACT_DIR already present:')
    print('   ZIP exists:', ZIP_PATH.exists(), '| Extracted folder exists:', EXTRACT_DIR.exists())


⚠️ Either ZIP_PATH does not exist or EXTRACT_DIR already present:
   ZIP exists: True | Extracted folder exists: True


## Step 2 — Constants

Change IMG_SIZE here if your original had another size.

In [5]:
# Constants (change if needed)
IMG_SIZE = 224  # Assumed 224x224 — change if your original used different input size
BATCH_SIZE = 16
SEED = 42
NUM_CLASSES = None  # will be inferred
DATA_DIR = Path('dataset_extracted')  # should contain train/ and test/ folders


## Step 3 — Create train/validation split (10%)
This cell will create a `validation` folder inside `dataset_extracted` if not already present. It moves a random 10% of training images per class to validation. Run once.

In [7]:
import shutil, random, os
from pathlib import Path

random.seed(SEED)

train_dir = DATA_DIR / 'train'
val_dir = DATA_DIR / 'val'
test_dir = DATA_DIR / 'test'

if not train_dir.exists():
    raise FileNotFoundError(f"Train directory not found: {train_dir.resolve()}")

if not val_dir.exists():
    print('Creating validation split (10%)...')
    for class_folder in train_dir.iterdir():
        if not class_folder.is_dir():
            continue
        images = list(class_folder.glob('*.*'))
        random.shuffle(images)
        k = max(1, int(len(images) * 0.10))
        dest = val_dir / class_folder.name
        dest.mkdir(parents=True, exist_ok=True)
        for img in images[:k]:
            shutil.move(str(img), str(dest / img.name))
    print('Validation split created at', val_dir.resolve())
else:
    print('Validation folder already exists — skipping split')

Validation folder already exists — skipping split


## Step 4 — Dataset summary (counts per split & per class)

In [9]:
from collections import defaultdict
def summarize_folder(path):
    summary = {}
    for c in sorted([p.name for p in path.iterdir() if p.is_dir()]):
        cnt = sum(1 for _ in (path / c).glob('*.*'))
        summary[c] = cnt
    return summary

for split in ['train','val','test']:
    p = DATA_DIR / split
    if p.exists():
        print(f"--- {split.upper()} ---")
        d = summarize_folder(p)
        total = sum(d.values())
        print(f"Total images: {total}")
        for k,v in d.items():
            print(f"  {k}: {v}") 
    else:
        print(f"Split not found: {p}")

--- TRAIN ---
Total images: 3638
  .ipynb_checkpoints: 0
  Damaged_building: 241
  Damaged_highway: 648
  Non-damaged_building: 1495
  Non-damaged_highway: 478
  damaged_buildings: 404
  debris: 372
--- VAL ---
Total images: 2748
  .ipynb_checkpoints: 0
  Damaged_building: 163
  Damaged_highway: 496
  Non-damaged_building: 1136
  Non-damaged_highway: 357
  damaged_buildings: 310
  debris: 286
--- TEST ---
Total images: 600
  Damaged_building: 100
  Damaged_highway: 100
  Non-damaged_building: 200
  Non-damaged_highway: 100
  debris: 100


## Step 5 — Data generators (augmentation on train; rescale on all)
Using `ImageDataGenerator` -> `flow_from_directory`.

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

DATA_DIR = Path("dataset_extracted")  # replace with your extracted folder name if different
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR / 'train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

val_generator = test_datagen.flow_from_directory(
    DATA_DIR / 'val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    DATA_DIR / 'test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

NUM_CLASSES = len(train_generator.class_indices)
print('Classes:', train_generator.class_indices)
print('NUM_CLASSES set to', NUM_CLASSES)

Found 3634 images belonging to 7 classes.
Found 2744 images belonging to 7 classes.
Found 600 images belonging to 5 classes.
Classes: {'.ipynb_checkpoints': 0, 'Damaged_building': 1, 'Damaged_highway': 2, 'Non-damaged_building': 3, 'Non-damaged_highway': 4, 'damaged_buildings': 5, 'debris': 6}
NUM_CLASSES set to 7


## Step 6 — Optimized CNN model
3 Conv blocks, BatchNorm, Dropout, GlobalAveragePooling. Good balance between capacity and overfitting control.

In [5]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Dense, Input

def build_model(img_size=IMG_SIZE, num_classes=NUM_CLASSES):
    inp = Input(shape=(img_size, img_size, 3))
    x = inp
    # Block 1
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.2)(x)
    # Block 2
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.3)(x)
    # Block 3
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.4)(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inp, out)
    return model

model = build_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 224, 224, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 112, 112, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 56, 56, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 323,623 (1.23 MB)

 Trainable params: 322,727 (1.23 MB)

 Non-trainable params: 896 (3.50 KB)

## Step 7 — Compile model & setup callbacks

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    ModelCheckpoint('best_damage_model.h5', save_best_only=True, monitor='val_loss'),
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7)
]

print('Callbacks ready')

Callbacks ready


## Step 8 — Train the model (Final run)
Adjust `epochs` as needed. This is the final training cell — run once.

In [ ]:
EPOCHS = 25  # adjust if you want more/less
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/25
 41/114 ━━━━━━━━━━━━━━━━━━━━ 20:44 17s/step - accuracy: 0.2466 - loss: 2.0480

## Step 9 — Plot training curves (accuracy & loss)

In [ ]:
import matplotlib.pyplot as plt

def plot_history(h):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(h.history['accuracy'], label='train_acc')
    plt.plot(h.history['val_accuracy'], label='val_acc')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(h.history['loss'], label='train_loss')
    plt.plot(h.history['val_loss'], label='val_loss')
    plt.title('Loss')
    plt.legend()
    plt.show()

plot_history(history)

## Step 10 — Evaluate on test set & show confusion matrix
This will compute classwise results using test_generator.

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Predict
y_pred_proba = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = test_generator.classes

print('Classification report:')
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

## Step 11 — Single image prediction example (one or two images is enough)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Update image_path to point to a sample image in test/CLASS/
sample_img = list((DATA_DIR/'test').rglob('*.*'))[0]  # first test image
img = image.load_img(sample_img, target_size=(IMG_SIZE, IMG_SIZE))
x = image.img_to_array(img)/255.0
x = np.expand_dims(x, axis=0)
pred = model.predict(x)
label = list(train_generator.class_indices.keys())[np.argmax(pred)]
print('Sample image:', sample_img)
print('Predicted label:', label)


## Step 12 — Save final model
`best_damage_model.h5` created by ModelCheckpoint will be the best model; you can also save the current weights.

In [ ]:
model.save('damage_model_final.h5')
print('Saved damage_model_final.h5')